## Astrometry.net solutions - 30-Jul-25

Craig Lage - Making a movie of 3I.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.geom import Point2D, SpherePoint, Angle, AngleUnit
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
import astropy.wcs
from astropy.io import fits
import pandas as pd
import pickle as pkl
from lsst.summit.utils.plotting import plot
from lsst.daf.butler import Butler
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
csv_filename = "/home/c/cslage/u/LSSTCam/data/3I_better_visits_radecs_forCraig.csv"
csv_data = pd.read_csv(csv_filename)


# The cell below makes the movie frames.

In [ ]:
filename = "/home/c/cslage/u/MTMount/mount_plots/wcsDict_3I.pkl"
with open(filename, 'rb') as f:
    wcsDict = pkl.load(f)
detector=30
filename = f"/home/c/cslage/u/LSSTCam/data/3I_Pixel_Locations_det{detector:02d}_16Jul25.txt"
data = np.loadtxt(filename, skiprows=1, dtype='str')

ra_center = 265.185
dec_center = -18.4562
stamp_size_x = 800
stamp_size_y = 300

#for expId in[2025071200522, 2025071200526, 2025071200530, 2025071200534, 2025071200538, 2025071200542, 2025071200562, 2025071200582]:
for n in range(len(data)):
    expId = int(data[n][0])
    key = f"{expId}_{detector:02d}"
    [wcs_header, wcs_from_fits] = wcsDict[key]
    thisRow = csv_data[csv_data['visit'] == expId]
    skyLocation = SkyCoord(ra_center*u.deg, dec_center*u.deg)
    pixel_center = wcs_from_fits.world_to_pixel(skyLocation)
    xmin = int(pixel_center[0] - stamp_size_x / 2.0)
    xmax = xmin + stamp_size_x
    ymin = int(pixel_center[1] - stamp_size_y / 4.0)
    ymax = ymin + stamp_size_y
    pixelX_3I = float(data[n][2]) - xmin
    pixelY_3I = float(data[n][3]) - ymin
    calexp = butler.get('preliminary_visit_image', detector=detector, visit=expId)
    fig = plt.figure(figsize=(16,16))
    arr = calexp.image.array
    myPlot = plot(arr[ymin:ymax, xmin:xmax], stretch='ccs', figure=fig)
    ax = myPlot.get_axes()[0]
    ax.scatter(pixelX_3I, pixelY_3I\
                ,color='red', marker='.', s=100)
    ax.set_title(f"{expId}")
    saveFilename = f"/home/c/cslage/u/LSSTCam/data/movie_3i/frame_{n:02d}.png"    
    plt.savefig(saveFilename)


In [ ]:
import os
import shlex, subprocess
dirName = "/home/c/cslage/u/LSSTCam/data/movie_3i"
movieName = "movie_3I.mp4"
command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -filter:v 'setpts=8.0*PTS' -vcodec libx264 -pix_fmt yuv420p -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()